In [2]:
import time, datetime
from datetime import datetime, timedelta
import csv
import calendar
import pandas as pd
import numpy.random as random
import sys

In [3]:
data = pd.read_csv("data.csv")

#multiply to get time in seconds
data['time'] = data['time']*8

In [4]:
timeSeries = list(data['time'])

reset = False
resetRows = []
for i in range(0,len(timeSeries)):
        if i > 0:
            if timeSeries[i] < timeSeries[i-1]:
                #recursively edit overflow
                timeSeries[i] = timeSeries[i-1] + 8
                #for debugging/logging
                reset = True
                resetRows.append(i)
if reset == True:
    print "edited due to overflow or reset"
    

#write data back into dataframe
data['time'] = timeSeries

edited due to overflow or reset


In [9]:
def singleSync(series):
    #obtain how many seconds elapsed 
    lastReading = data[series][len(data)-1]

    calculatedStartDate = datetime.now()- timedelta(seconds=int(lastReading))
    #load times as timedelta
    timedeltas = pd.to_timedelta(data[series],unit='s')
    columnName = str('realtime - ' + series)
    #adjust in time since epoch the starting time of the loggings
    data[columnName] = calculatedStartDate + timedeltas

    #time string formatting
    data[columnName] = pd.to_datetime(pd.DatetimeIndex(data[columnName]).strftime('%X %D'))

In [10]:
singleSync('time')
#generate some noise
def noise():
    return random.randint(-2,2) * random.rand()

timeSeries = list(data['time'])
#generate a fake user-inputted start date with some error built-in (for testing time correction purposes only

for i in range (0,len(timeSeries)):
        #if i == 0:
         #       timeSeries[i] = timeSeries[i] + noise()
        if i > 0:
                timeSeries[i] = timeSeries[i-1] + 8 + noise()

data['noisytime'] = timeSeries

singleSync('noisytime')


In [11]:
def getUserStartDate():
    while True:
        #userIn = raw_input("Type Date: mm/dd/yy: ")
        userDateInput = raw_input("enter the day the sensor was turned on and placed in the nest as mm/dd/yyyy ")
        try:
            date = datetime.strptime(userDateInput, "%m/%d/%Y")
            #date = datetime.date(year=date.year, month = date.month, day=date.day)
        except ValueError as e:
            print "Invalid Format: {0}".format(e)
        else:
            return date

def getUserStartTime():
    while True:
        #do the biologists use 24hr time?
        userTimeInput = raw_input("enter the time the sensor was turned on and placed in the nest as 24-Hour time in the format hr:min (e.x. 16:32): ")
        try:
            time = datetime.strptime(userTimeInput + ":00", "%X")
            #time = timedelta(hours=time.hour, minutes=time.minute, seconds=0)
        except ValueError as e:
            print "Invalid Format: {0}".format(e)
        else:
            return time

In [12]:
#from http://code.activestate.com/recipes/577058/
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is True for "yes" or False for "no".
    """
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [yes/no] "
    elif default == "yes":
        prompt = " [yes/no] "
    elif default == "no":
        prompt = " [yes/no] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")

In [87]:
def valiDate():
    date = getUserStartDate()
    time = getUserStartTime()
    userStartDateTime = datetime(year=date.year, month=date.month, day=date.day, hour=time.hour, minute= time.minute, second=0)
    print "your date is: " + str(userStartDateTime)
    userVerify = query_yes_no("is this correct? ")
    if userVerify == False:
        valiDate()
    if userVerify == True:
        print "synchronizing time now..."
        return userStartDateTime


def DoubleSync(series):
    columnName = str('doubleSyncRealTime - ' + series)
    #if the user start date doesn't match the singleSync start date, do I adjust by taking the
    #currentTime - userStartTime / number of readings (not preserving 8-sec logging) or by doing
    #userStartTime + 8n?

    #% will adjust syncing approach based on the percentage off the data is.
    tolerance = .1
    userStartDateTime = valiDate()
    timeElapsed = data[series][len(data[series])-1] - data[series][0]
    if userStartDateTime - data[series][0] !=0:
        percentOff = abs((userStartDateTime - data[series][0])/(data[series][len(data[series])-1] - data[series][0]))
        print percentOff
        if percentOff > tolerance:
            #userStartTime + 8n
            data['doublesync'] = pd.date_range(freq=pd.DateOffset(seconds=8),start=userStartDateTime,periods=len(data[series]))
            #for i in range(0, len(data[series])-1):
               # print userStartDateTime + timedelta(seconds=8*i)
                #data['doublesync'].append(userStartDateTime + timedelta(0,0, 8*i ))
            data.head()
        elif percentOff < tolerance:
            #print "wow, you're actually pretty accurate"
            #interval = 8 seconds +/- some number of microseconds
            interval = (data[series][len(data[series])-1] - userStartDateTime)/(len(data[series]) - 1)
            print str(interval) + "seconds"
            data['doublesync'] = pd.date_range(start=userStartDateTime, periods= len(data[series]), freq=pd.DateOffset(seconds=interval.seconds, microseconds=interval.microseconds))
            

#data['doublesync'].append(datetime(2017,10,27,8,59,0))
DoubleSync('realtime - noisytime')

enter the day the sensor was turned on and placed in the nest as mm/dd/yyyy 2/25/2017
enter the time the sensor was turned on and placed in the nest as 24-Hour time in the format hr:min (e.x. 16:32): 14:14
your date is: 2017-02-25 14:14:00
is this correct?  [yes/no] yes
synchronizing time now...
0.0975055130289
0 days 00:00:08.448326seconds


In [90]:
data['plotting'], data['realtime - noisytime']

(0    2017-02-25 14:14:00.000000
 1    2017-02-25 14:14:08.448326
 2    2017-02-25 14:14:16.896652
 3    2017-02-25 14:14:25.344978
 4    2017-02-25 14:14:33.793304
 5    2017-02-25 14:14:42.241630
 6    2017-02-25 14:14:50.689956
 7    2017-02-25 14:14:59.138282
 8    2017-02-25 14:15:07.586608
 9    2017-02-25 14:15:16.034934
 10   2017-02-25 14:15:24.483260
 11   2017-02-25 14:15:32.931586
 12   2017-02-25 14:15:41.379912
 13   2017-02-25 14:15:49.828238
 14   2017-02-25 14:15:58.276564
 15   2017-02-25 14:16:06.724890
 16   2017-02-25 14:16:15.173216
 17   2017-02-25 14:16:23.621542
 18   2017-02-25 14:16:32.069868
 19   2017-02-25 14:16:40.518194
 20   2017-02-25 14:16:48.966520
 21   2017-02-25 14:16:57.414846
 22   2017-02-25 14:17:05.863172
 23   2017-02-25 14:17:14.311498
 24   2017-02-25 14:17:22.759824
 25   2017-02-25 14:17:31.208150
 26   2017-02-25 14:17:39.656476
 27   2017-02-25 14:17:48.104802
 28   2017-02-25 14:17:56.553128
 29   2017-02-25 14:18:05.001454
          

In [86]:
data['plotting'].dtype

dtype('<M8[ns]')

In [92]:
def plot_series(x, y):
    fig, ax = plt.subplots()
    ax.plot_date(x, y, fmt='g--') # x = array of dates, y = array of numbers

    fig.autofmt_xdate()
    plt.grid(True)
    plt.show()

plot_series('plotting','humidity')

AttributeError: 'numpy.string_' object has no attribute 'toordinal'

In [36]:
data['realtime - time'][2].dtype

AttributeError: 'Timestamp' object has no attribute 'dtype'

In [35]:
data['doublesync'].dtype

AttributeError: 'Timestamp' object has no attribute 'dtype'